# Demo: Serenity Derivatives API - Option Pricing

Serenity builds in sophisticated option and rates analytics as part of its core offering, and these functions
are all exposed via the API. This notebook shows how you can use it to price European options on BTC and ETH.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
from datetime import datetime, timedelta
from typing import List, Dict
from uuid import UUID, uuid4

from time import sleep
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools

from serenity_types.pricing.derivatives.rates.yield_curve import YieldCurveVersion
from serenity_types.pricing.derivatives.options.valuation import (
    DiscountingMethod, 
    MarketDataOverride, 
    YieldCurveOverride,
    OptionValuationRequest, 
    OptionValuation)
from serenity_types.pricing.derivatives.options.volsurface import (
    InterpolatedVolatilitySurface, VolModel, DiscountingMethod, ProjectionMethod
)

from serenity_sdk.renderers.derivatives.request_helpers import (
    run_compute_option_valuations, 
    run_multiple_option_valuation_requests
)
from serenity_sdk.renderers.derivatives.widget_tools import OptionChooser
from serenity_sdk.renderers.derivatives.table_plot import (
    YieldCurveTablePlot, 
    VolatilitySurfaceTablePlot, 
    OptionValuationResultTablePlot, 
    plot_valuation_results,
    plot_bumped_pv
)
from serenity_sdk.renderers.derivatives.overrides import apply_option_valuation_overrides
from serenity_sdk.renderers.derivatives.converters import convert_object_dict_to_df, convert_object_list_to_df

# set default plot parameters
plt.rcParams['font.size'] = '16'

# create an alias to the api
api = connect_widget.get_api()

# Load samples of pre-defined option instruments

### Load & peek at samples of predefined options

OptionChooser is a helper function. It wraps 

* `api.pricer().get_supported_underliers()`: get the supported underlier information (i.e. BTC and ETH)
* `api.pricer().get_supported_options()`: get a list of pre-defined options from Deribit. 

In [ ]:
option_chooser = OptionChooser(api)
option_chooser.data.head(3)

## Select the option to use as a base line

In [ ]:
print('Select an option to play with')
display(option_chooser.get_widget_to_display())

In [ ]:
# Show the details of the option selected
the_default_option_info = option_chooser.get_selected_option()
pd.Series(the_default_option_info)

# Option Valuation

Helper functions: Throughout this notebook, we use the following two helper functions that wraps `api.pricer().compute_option_valuations` to help displaying valuation results. 
* `run_compute_option_valuations`
* `run_multiple_option_valuation_requests`

For example, API's `compute_option_valuations` take an array of option valuations. Each option valuation is identified using a unique uuid. To help visualising option valuation results, `run_compute_option_valuations` wraps `compute_option_valuations` and takes a dictionary of option valuations with keys set by users. The valuation results are unpacked and put into a Pandas dataframe with columns corresponding to the keys. 

## Define demo option valuations

* Use the asset_id (uuid) of the pre-defined option to construct a option valuation object
* Also, create its replica using option attributues


In [ ]:
# The default option valuation object
the_default_optval = OptionValuation(
    valuation_id=str(uuid4()),
    qty = 10, 
    option_asset_id=the_default_option_info['asset_id'])

# Replicate the same option using the attributes of the prefeined option earlier
the_replica_optval = OptionValuation(
    valuation_id=str(uuid4()),
    qty = the_default_optval.qty,  # use the qty from the default option
    underlier_asset_id=the_default_option_info['underlier_asset_id'],
    strike=the_default_option_info['strike_price'],
    expiry=the_default_option_info['expiry_datetime'],
    option_type=the_default_option_info['option_type'],
    option_style=the_default_option_info['option_style'],
    contract_size=the_default_option_info['contract_size']
    )

In [ ]:
# use the predefined option (default option)
demo0_optvals = {'predefined': the_default_optval}
convert_object_dict_to_df(demo0_optvals)

## Real-time and Historical valuation modes

1. Real-time valuation model: With `as_of_time` left empty. 
1. Historical valuation mode: With `as_of_time` specified. 

### Real-time Valuation Model: Leave `as_of_time`` as empty.

In [ ]:
# Run several times, sleep a few seconds after each run. 
a_res = {}
num_calls, sleep_sec= 10, 1

# Try different discounting & projection methods
discounting_method = DiscountingMethod.CURVE
projection_method = ProjectionMethod.CURVE
print(f"Start making {num_calls} calls, {sleep_sec} seconds apart. Please, wait at least {num_calls * sleep_sec} seconds.")
for j in range(num_calls):
    now_ts = datetime.utcnow()
    res_table = run_compute_option_valuations(api, demo0_optvals, discounting_method=discounting_method, projection_method=projection_method)
    a_res[now_ts] = res_table.iloc[:,0]
    if j < num_calls-1:
        sleep(sleep_sec)

print('Expected Behaviour: Results should change over time.')
res_df = pd.DataFrame(a_res)
res_df

In [ ]:
# Let's plot some key values
plot_valuation_results(res_df)

### Historical Valuation Mode: Specify `as_of_time`

In [ ]:
# Run several times, separated by a few second
# We should get the same results. 

as_of_time = datetime.utcnow()
a_res = {}
num_calls, sleep_sec = 2, 2.5
for j in range(num_calls):
    res_table = run_compute_option_valuations(api, demo0_optvals, as_of_time=as_of_time)
    a_res[f"run_{j}"] = res_table.iloc[:,0]
    if j < num_calls-1:
        print(f'sleeping {sleep_sec} seconds.'), sleep(sleep_sec)

print('Expected Behaviour: They should return the exactly the same results since "as_of_time" is specified.')

pd.DataFrame(a_res)

## Different Valuation Modes
In addition to real-time and historical modes, we allow other modes by specifying `projection_method` and `discounting_method`. 

At this point, some of the supported combinations are demonstrated below. 

In [ ]:
my_options = [the_default_optval]
as_of_time = datetime.utcnow()

val_requests = {
    'real-time (default)': OptionValuationRequest(options=my_options), 
    'historical (default)' : OptionValuationRequest(options=my_options, as_of_time=as_of_time),
    'real-time (PM=F/DM=S)': OptionValuationRequest(options=my_options, projection_method=ProjectionMethod.FUTURES, discounting_method=DiscountingMethod.SELF_DISCOUNTING),
    'real-time (PM=F/DM=C)': OptionValuationRequest(options=my_options, projection_method=ProjectionMethod.FUTURES, discounting_method=DiscountingMethod.CURVE),
    'real-time (PM=C/DM=S)': OptionValuationRequest(options=my_options, projection_method=ProjectionMethod.CURVE, discounting_method=DiscountingMethod.SELF_DISCOUNTING),
    'real-time (PM=C/DM=C)': OptionValuationRequest(options=my_options, projection_method=ProjectionMethod.CURVE, discounting_method=DiscountingMethod.CURVE),
    'historical (PM=C/DM=S)': OptionValuationRequest(options=my_options, as_of_time=as_of_time, projection_method=ProjectionMethod.CURVE, discounting_method=DiscountingMethod.SELF_DISCOUNTING),
    'historical (PM=C/DM=C)': OptionValuationRequest(options=my_options, as_of_time=as_of_time, projection_method=ProjectionMethod.CURVE, discounting_method=DiscountingMethod.CURVE)
}

res_table, _, _ = run_multiple_option_valuation_requests(api, val_requests)
res_table

## Option Valuation using attributes (strikes, expiry, etc)
Consider the predefined option and its replica using the same attributes

In [ ]:
demo1_optvals = {'predefined':the_default_optval, 'predefined_replica':the_replica_optval}

# show option vals
convert_object_dict_to_df(demo1_optvals)

In [ ]:
res_table = run_compute_option_valuations(api, demo1_optvals)
print('Expected Behaviour: Results should be identifical.')
res_table


## Option Valuation using own attributes (e.g. for OTC options)

In [ ]:
symbol_to_asset_id = convert_object_list_to_df(api.pricer().get_supported_underliers()).set_index('native_symbol')['asset_id']
symbol_to_asset_id

In [ ]:
# Create an OTC option by specifying the attributes
my_otc_optval_call = OptionValuation(
    valuation_id=str(uuid4()),
    qty = 10.0,  # use the qty from the default option
    underlier_asset_id=symbol_to_asset_id['BTC'],
    strike=22000.0, # strike in BTC-USD
    expiry=datetime(2024,1,31, 8, 0, 0), # expiry (in UTC)
    option_type='CALL', # 'CALL' or 'PUT'
    option_style='EUROPEAN', # currently 'EUROPEAN' only
    contract_size=1.0
    )

my_otc_optval_put = OptionValuation(
    valuation_id=str(uuid4()),
    qty = -10.0,  # use the qty from the default option
    underlier_asset_id=symbol_to_asset_id['BTC'],
    strike=18000.0, # strike in BTC-USD
    expiry=datetime(2024,1,31, 8, 0, 0), # expiry (in UTC)
    option_type='PUT', # 'CALL' or 'PUT'
    option_style='EUROPEAN', # currently 'EUROPEAN' only
    contract_size=1.0
    )

# create 
otc_optvals = {'my_call':my_otc_optval_call, 'my_put': my_otc_optval_put}
# show option vals
convert_object_dict_to_df(otc_optvals)

In [ ]:
# Run
res_table = run_compute_option_valuations(api, otc_optvals)
res_table

## With Market Data Overrides

Let's override market data and see the impact on the present value. 
To this end, 
* Pick an option with the default market data as the 'base' case. 
* Create a collection of new option valuations with market data bumps over a set of bump sizes. 
* Send the base case and the bumped valuations to API to get the PVs back. 

To illustrate the results, 
* Calculate the PV impact of the bumps, i.e. PNL(bumped) = PV(bumped) - PV(base)
* Plot PNLs over bumps.
* Overlay Taylor approximations using Greeks of the base-case valuation. 

### Keep spot & vol base values
For realistic demos below, let's keep the spot & vol level

In [ ]:
spot_base = res_table.loc['spot_price'].iloc[0]  # get a hint on the spot price level from the previous run
vol_base = res_table.loc['iv'].iloc[0]

### Define a set of option valuations with spot bumps and send them to API

In [ ]:
# spot price dumpbs
bumps_in_perc_multiplicative = [-20.0, -10.0, -5.0, -2.5, -1.0, 0.0, +1.0, +2.5, +5.0, +10.0, +20.0]
# market data bump
spot_bumps = {f'{sb}':spot_base*sb/100 for sb in bumps_in_perc_multiplicative}

# pick the base option to bump
base_optval = the_replica_optval.copy()

# dictionary of option valuations with market data bumped
spot_bumps_optvals = {'base': base_optval}
for sb_key, sb_val in spot_bumps.items():
    spot_bumps_optvals[f'spot_bump_{sb_key}%'] = apply_option_valuation_overrides(base_optval, spot_override=sb_val)

# show option valuations
convert_object_dict_to_df(spot_bumps_optvals)

In [ ]:
# Run 'compute_option_valuations'
res_table = run_compute_option_valuations(api, spot_bumps_optvals)
res_table

Let's plot. The Taylor expansion using the delta (and gamma) from the base option should give a good approximation of the spot bump/revals

In [ ]:
plot_bumped_pv(res_table, the_replica_optval.qty, spot_bumps, 'spot', 'delta', 'gamma')

### Vol Bumps

In [ ]:
# vol bmps 
bumps_in_perc = [-20.0, -10.0, -5.0, -2.5, -1.0, 0.0, +1.0, +2.5, +5.0, +10.0, +20.0]
# market data bump
vol_bumps = {f'{sb}':sb/100 for sb in bumps_in_perc}

# pick the base option to bump
base_optval = the_replica_optval.copy()

# dictionary of option valuations with market data bumped
vol_bumps_optvals = {'base': base_optval}
for sb_key, sb_val in vol_bumps.items():
    vol_bumps_optvals[f'vol_bump_{sb_key}%'] = apply_option_valuation_overrides(base_optval, vol_override=sb_val)

# show option valuations
convert_object_dict_to_df(vol_bumps_optvals)

In [ ]:
# Run 'compute_option_valuations'
res_table = run_compute_option_valuations(api, vol_bumps_optvals)
res_table

Let's plot. The PnL profile should be well approximated by the first-order Taylor approximation, i.e. 

vega(base case) * (vol bump).

In [ ]:
plot_bumped_pv(res_table, the_replica_optval.qty, vol_bumps, 'vol', 'vega')

## Replacements

In [ ]:
spot_replacement = spot_base * 1.2
vol_replacement = vol_base * 1.3

my_optval = apply_option_valuation_overrides(the_replica_optval, 
    spot_override=spot_replacement, 
    spot_is_bump=False, 
    vol_override=vol_replacement,
    vol_is_bump=False)
replacement_optvals = {'base': the_replica_optval, 'replacement': my_optval}
convert_object_dict_to_df(replacement_optvals)

In [ ]:
# Run 'compute_option_valuations'
res_table = run_compute_option_valuations(api, replacement_optvals)
print('Expected Result: spot_price and iv are replaced as specified.')
res_table

## Options over Expiries

So far, we considered an option with a specific expiry datetime and a strike. Now, let's consider 'ATM' options over a set of expiries. Here, ATM in the sense that the strike is the same to the spot price. 

In [ ]:
# Expiries
now_utc = datetime.utcnow()
expiry_times = {d:now_utc + timedelta(d) for d in [0.01, 1, 7, 14, 30, 60, 90, 180, 360, 720]} # in days

base_optval = the_replica_optval.copy()
expiries_optvals = {}
for d, ex in expiry_times.items():
    expiries_optvals[f"{d}-days"] = apply_option_valuation_overrides(base_optval, strike_override=spot_base, expiry_override=ex)

# show option valuations
convert_object_dict_to_df(expiries_optvals)


In [ ]:
# Run 'compute_option_valuations'
as_of_time = now_utc - timedelta(hours=0)
projection_method=ProjectionMethod.CURVE
discounting_method=DiscountingMethod.CURVE
res_table = run_compute_option_valuations(
    api, expiries_optvals, 
    as_of_time = as_of_time, 
    projection_method=projection_method, discounting_method=discounting_method)   
res_table

Let's plot some fields from the valuation results over expiries

In [ ]:
plot_valuation_results(res_table, expiry_times.values())

# END